In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
from torchvision import models
from skimage import io, color, transform
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
import multiprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Path to the folders containing real and fake images
original_folder_path = "/content/gdrive/MyDrive/celebdf/original-cropped-images"
synthetic_folder_path = "/content/gdrive/MyDrive/celebdf/synthetic-cropped-images"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
def load_and_preprocess_images(folder):
    images = []

    count=0
    for filename in os.listdir(folder):
      img_path = os.path.join(folder, filename)
      img = io.imread(img_path)
      img_gray = color.rgb2gray(img)
      img_resized = transform.resize(img_gray, (64, 64))  # Resize to a consistent size

      # Convert the NumPy array to a PyTorch tensor
      img_tensor = torch.tensor(img_resized, dtype=torch.float32).to(device)
      images.append(img_tensor)

      count+=1
      print(count)

    return images

In [ ]:
real_images = load_and_preprocess_images(original_folder_path)

In [ ]:
fake_images = load_and_preprocess_images(synthetic_folder_path)

In [ ]:
# Create labels (0 for real, 1 for fake)
real_labels_tensor = torch.zeros(len(real_images), dtype=torch.float32).to(device)
fake_labels_tensor = torch.ones(len(fake_images), dtype=torch.float32).to(device)

In [ ]:
real_images_tensor = torch.stack(real_images)
fake_images_tensor = torch.stack(fake_images)

In [ ]:
# Combine real and fake data
all_images = torch.cat((real_images_tensor, fake_images_tensor), dim=0)
all_labels = torch.cat((real_labels_tensor, fake_labels_tensor), dim=0)

In [ ]:
all_images_numpy = all_images.cpu().numpy()
all_labels_numpy = all_labels.cpu().numpy()

# Split the dataset into training and testing sets
X_train_numpy, X_test_numpy, y_train_numpy, y_test_numpy = train_test_split(
    all_images_numpy, all_labels_numpy, test_size=0.2, random_state=42
)

In [ ]:
# Convert the NumPy arrays back to PyTorch tensors (and move to GPU if available)
X_train = torch.tensor(X_train_numpy, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test_numpy, dtype=torch.float32).to(device)
lbp_train_labels = torch.tensor(y_train_numpy, dtype=torch.float32).to(device)
lbp_test_labels = torch.tensor(y_test_numpy, dtype=torch.float32).to(device)

## LBP + NN

In [ ]:
# Parameters for LBP
radius = 1
n_points = 8 * radius

# Extract LBP features using PyTorch and GPU
def extract_lbp_features(images):
    features = []
    for img in images:
        img = img.squeeze(0).cpu().numpy()  # Convert the tensor to NumPy and remove the batch dimension
        lbp_img = local_binary_pattern(img, n_points, radius, method='uniform')
        hist, _ = np.histogram(lbp_img.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-8)
        features.append(hist)
    return features

In [ ]:
train_lbp_features = extract_lbp_features(X_train)
test_lbp_features = extract_lbp_features(X_test)
train_lbp_features = torch.tensor(train_lbp_features, dtype=torch.float32).to(device)
test_lbp_features = torch.tensor(test_lbp_features, dtype=torch.float32).to(device)

<ipython-input-14-b5c973d1c79d>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_lbp_features = torch.tensor(train_lbp_features, dtype=torch.float32).to(device)


In [ ]:
train_images = torch.tensor(X_train_numpy, dtype=torch.float32).unsqueeze(1).to(device)
test_images = torch.tensor(X_test_numpy, dtype=torch.float32).unsqueeze(1).to(device)
train_labels = torch.tensor(y_train_numpy, dtype=torch.float32).to(device)
test_labels = torch.tensor(y_test_numpy, dtype=torch.float32).to(device)

In [ ]:
# torch.save(train_lbp_features,"/content/gdrive/MyDrive/saved-tensors/train_lbp_features.pt")
torch.save(test_lbp_features,"/content/gdrive/MyDrive/saved-tensors/test_lbp_features.pt")
torch.save(train_images,"/content/gdrive/MyDrive/saved-tensors/train_images.pt")
torch.save(train_labels,"/content/gdrive/MyDrive/saved-tensors/train_labels.pt")
torch.save(test_images,"/content/gdrive/MyDrive/saved-tensors/test_images.pt")
torch.save(test_labels,"/content/gdrive/MyDrive/saved-tensors/test_labels.pt")

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, input_size):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5)
        )
        self.fc4 = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5)
        )
        self.fc5 = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.5)
        )
        self.fc6 = nn.Sequential(
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input if not already flattened
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        return x

In [ ]:
train_lbp_features.shape

torch.Size([27800, 10])

In [ ]:
# Define the input feature dimension
feature_dim = train_lbp_features.shape[1]

# Create an instance of the CustomModel
model = CustomModel(feature_dim)

model.to(device)

In [ ]:
# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

In [ ]:
# Define the number of training epochs
num_epochs = 10
batch_size=32

# Convert X_train_features and y_train to PyTorch tensors and create a DataLoader
train_dataset = TensorDataset(train_lbp_features, lbp_train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0

    for inputs, labels in train_loader:
        # Move inputs and labels to the GPU if available
        inputs, labels = inputs.to(device), labels.to(device)
        labels=labels.unsqueeze(1)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Calculate and print the average loss for the epoch
    average_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f}")

Epoch [1/10] - Loss: 0.5852
Epoch [2/10] - Loss: 0.5467
Epoch [3/10] - Loss: 0.5387
Epoch [4/10] - Loss: 0.5356
Epoch [5/10] - Loss: 0.5339
Epoch [6/10] - Loss: 0.5314
Epoch [7/10] - Loss: 0.5293
Epoch [8/10] - Loss: 0.5303
Epoch [9/10] - Loss: 0.5281
Epoch [10/10] - Loss: 0.5298


In [ ]:
# Set the model to evaluation mode
model.eval()

# Calculate predictions
with torch.no_grad():
    predictions = model(test_lbp_features)

# Calculate the test loss
lbp_test_labels=lbp_test_labels.unsqueeze(1)
test_loss = criterion(predictions, lbp_test_labels)

# Calculate the test accuracy
predictions_binary = (predictions >= 0.5).float()  # Convert probabilities to binary predictions
correct_predictions = (predictions_binary == lbp_test_labels).sum().item()
total_samples = len(lbp_test_labels)
test_accuracy = correct_predictions / total_samples

print("Test loss:", test_loss.item())
print("Test accuracy:", test_accuracy)

Test loss: 0.523662269115448
Test accuracy: 0.7571223021582734


## Resnet50 + LBP

In [ ]:
# Load a pre-trained ResNet model
resnet50 = models.resnet101(pretrained=True)

# Change the input layer to accept single-channel (grayscale) images
resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=2, stride=1, padding=1, bias=False)

# Freeze all layers except the final classification layer
for param in resnet50.parameters():
    param.requires_grad = False
num_features = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_features, 2)  # 2 output classes for binary classification

# Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.fc.parameters(), lr=0.1)

resnet50.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, input_size):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5)
        )
        self.fc4 = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5)
        )
        self.fc5 = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.5)
        )
        self.fc6 = nn.Sequential(
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input if not already flattened
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        return x

In [ ]:
# Define the input feature dimension
feature_dim = train_lbp_features.shape[1]

# Create an instance of the CustomModel
custom_model = CustomModel(feature_dim)

custom_model.to(device)

CustomModel(
  (fc1): Sequential(
    (0): Linear(in_features=10, out_features=512, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5, inplace=False)
  )
  (fc2): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5, inplace=False)
  )
  (fc3): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5, inplace=False)
  )
  (fc4): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5, inplace=False)
  )
  (fc5): Sequential(
    (0): Linear(in_features=64, out_featu

In [ ]:
# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(custom_model.parameters(), lr=0.1)  # Adam optimizer with a learning rate of 0.001

In [ ]:
class CombinedModel(nn.Module):
    def __init__(self, resnet_model, custom_model):
        super(CombinedModel, self).__init__()
        self.resnet_model = resnet_model
        self.custom_model = custom_model
        self.fc_final = nn.Sequential(
            nn.Linear(3, 1),
            nn.Sigmoid()
        )
    def forward(self, x_resnet, x_custom):
        # Forward pass through the ResNet model
        resnet_output = self.resnet_model(x_resnet)

        # Forward pass through the Custom model
        custom_output = self.custom_model(x_custom)

        # You can combine the outputs in various ways depending on your task.
        # For example, you can concatenate them:
        combined_output = torch.cat((resnet_output.to(device), custom_output.to(device)), dim=1)

        modified_output = self.fc_final(combined_output.to(device))

        return modified_output.to(device)

In [ ]:
combined_model = CombinedModel(resnet50,custom_model)
combined_model.to(device)

CombinedModel(
  (resnet_model): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(

In [ ]:
# Define the number of training epochs
num_epochs = 20
batch_size=32

# Convert X_train_features and y_train to PyTorch tensors and create a DataLoader
lbp_train_dataset = TensorDataset(train_lbp_features, lbp_train_labels)
lbp_train_loader = DataLoader(lbp_train_dataset, batch_size=batch_size, shuffle=False)
lbp_test_dataset = TensorDataset(test_lbp_features, lbp_test_labels)
lbp_test_loader = DataLoader(lbp_test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# resnet data
train_images = torch.tensor(X_train_numpy, dtype=torch.float32).unsqueeze(1).to(device)
test_images = torch.tensor(X_test_numpy, dtype=torch.float32).unsqueeze(1).to(device)
train_labels = torch.tensor(y_train_numpy, dtype=torch.float32).to(device)
test_labels = torch.tensor(y_test_numpy, dtype=torch.float32).to(device)

# Create DataLoader for training and testing datasets
rn_train_dataset = TensorDataset(train_images, train_labels)
rn_train_loader = DataLoader(rn_train_dataset, batch_size=32, shuffle=False)
rn_test_dataset = TensorDataset(test_images, test_labels)
rn_test_loader = DataLoader(rn_test_dataset, batch_size=32, shuffle=False)

In [ ]:
for epoch in range(num_epochs):
    combined_model.train()  # Set the model to training mode
    total_loss = 0.0

    for (batch_data_resnet, labels_resnet), (batch_data_custom, labels_custom) in zip(rn_train_loader, lbp_train_loader):
        optimizer.zero_grad()  # Zero the gradients
        batch_data_resnet,batch_data_custom=batch_data_resnet.to(device),batch_data_custom.to(device)
        # labels_resnet.unsqueeze(1)
        # labels_custom.unsqueeze(1)
        # labels_resnet = labels_resnet.view(-1).float().to(device)
        # Forward pass through the combined model
        combined_output = combined_model(batch_data_resnet, batch_data_custom)
        # Compute the loss
        loss = criterion(combined_output, labels_resnet.view(-1, 1).float().to(device))  # You can choose which labels to use

        # Backpropagation
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Calculate and print average training loss for this epoch
    avg_train_loss = total_loss / len(rn_train_loader)  # You can use either loader for length
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {avg_train_loss:.4f}")

Epoch [1/20] - Training Loss: 0.6869
Epoch [2/20] - Training Loss: 0.6868
Epoch [3/20] - Training Loss: 0.6868
Epoch [4/20] - Training Loss: 0.6868
Epoch [5/20] - Training Loss: 0.6868
Epoch [6/20] - Training Loss: 0.6868
Epoch [7/20] - Training Loss: 0.6868
Epoch [8/20] - Training Loss: 0.6868
Epoch [9/20] - Training Loss: 0.6868
Epoch [10/20] - Training Loss: 0.6868
Epoch [11/20] - Training Loss: 0.6868
Epoch [12/20] - Training Loss: 0.6868
Epoch [13/20] - Training Loss: 0.6868
Epoch [14/20] - Training Loss: 0.6868
Epoch [15/20] - Training Loss: 0.6868
Epoch [16/20] - Training Loss: 0.6868
Epoch [17/20] - Training Loss: 0.6868
Epoch [18/20] - Training Loss: 0.6868
Epoch [19/20] - Training Loss: 0.6868
Epoch [20/20] - Training Loss: 0.6868


In [ ]:
combined_model.eval()  # Set the model to evaluation mode

# Initialize variables to store predictions and ground truth labels
predictions = []
true_labels = []

with torch.no_grad():
    for (batch_data_resnet, labels_resnet), (batch_data_lbp, labels_lbp) in zip(rn_test_loader, lbp_test_loader):
        batch_data_resnet=batch_data_resnet.to(device)
        batch_data_lbp=batch_data_lbp.to(device)
        labels_resnet=labels_resnet.to(device)
        labels_resnet.unsqueeze(1)
        outputs = combined_model(batch_data_resnet,batch_data_lbp)
        _, predicted = torch.max(outputs, 1)  # Get the class with the highest probability
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels_resnet.cpu().numpy())

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.2446043165467626


# RESNET 18

In [ ]:
# Convert the NumPy arrays back to PyTorch tensors (and move to GPU if available)
train_images = torch.tensor(X_train_numpy, dtype=torch.float32).unsqueeze(1).to(device)
test_images = torch.tensor(X_test_numpy, dtype=torch.float32).unsqueeze(1).to(device)
train_labels = torch.tensor(y_train_numpy, dtype=torch.float32).to(device)
test_labels = torch.tensor(y_test_numpy, dtype=torch.float32).to(device)

In [ ]:
train_images.shape

torch.Size([27800, 1, 64, 64])

In [ ]:
# Create DataLoader for training and testing datasets
train_dataset = TensorDataset(train_images, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
test_dataset = TensorDataset(test_images, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Load a pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Change the input layer to accept single-channel (grayscale) images
model.conv1 = nn.Conv2d(1, 64, kernel_size=2, stride=1, padding=1, bias=False)

# Freeze all layers except the final classification layer
for param in model.parameters():
    param.requires_grad = False
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 output classes for binary classification

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

model.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs=inputs.to(device)
        labels = labels.type(torch.LongTensor).to(device)
        labels.unsqueeze(1);
        labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/10, Loss: 0.4898523797202851
Epoch 2/10, Loss: 0.45746622696707523
Epoch 3/10, Loss: 0.4497464854951832
Epoch 4/10, Loss: 0.4434650909537139
Epoch 5/10, Loss: 0.4393039422047563
Epoch 6/10, Loss: 0.4416201966026984
Epoch 7/10, Loss: 0.44141575543186334
Epoch 8/10, Loss: 0.44386070670069155
Epoch 9/10, Loss: 0.4399309666576539
Epoch 10/10, Loss: 0.4413062349836352


In [ ]:
model.eval()  # Set the model to evaluation mode

# Initialize variables to store predictions and ground truth labels
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        labels.unsqueeze(1)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the class with the highest probability
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8007194244604317


# RESNET 34

In [ ]:
# Convert the NumPy arrays back to PyTorch tensors (and move to GPU if available)
train_images = torch.tensor(X_train_numpy, dtype=torch.float32).unsqueeze(1).to(device)
test_images = torch.tensor(X_test_numpy, dtype=torch.float32).unsqueeze(1).to(device)
train_labels = torch.tensor(y_train_numpy, dtype=torch.float32).to(device)
test_labels = torch.tensor(y_test_numpy, dtype=torch.float32).to(device)

In [ ]:
train_images.shape

torch.Size([27800, 1, 64, 64])

In [ ]:
# Create DataLoader for training and testing datasets
train_dataset = TensorDataset(train_images, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
test_dataset = TensorDataset(test_images, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Load a pre-trained ResNet model
model = models.resnet34(pretrained=True)

# Change the input layer to accept single-channel (grayscale) images
model.conv1 = nn.Conv2d(1, 64, kernel_size=2, stride=1, padding=1, bias=False)

# Freeze all layers except the final classification layer
for param in model.parameters():
    param.requires_grad = False
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 output classes for binary classification

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

model.to(device)

In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs=inputs.to(device)
        labels = labels.type(torch.LongTensor).to(device)
        labels.unsqueeze(1);
        labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/10, Loss: 0.48464311746788247
Epoch 2/10, Loss: 0.4512580076727686
Epoch 3/10, Loss: 0.4497603009121602
Epoch 4/10, Loss: 0.44812367518697155
Epoch 5/10, Loss: 0.4431748813397591
Epoch 6/10, Loss: 0.44179207573551577
Epoch 7/10, Loss: 0.4422336682179992
Epoch 8/10, Loss: 0.4478660127784632
Epoch 9/10, Loss: 0.44458653202539755
Epoch 10/10, Loss: 0.44304224617256255


In [ ]:
model.eval()  # Set the model to evaluation mode

# Initialize variables to store predictions and ground truth labels
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        labels.unsqueeze(1)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the class with the highest probability
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8061870503597123


# RESNET 50

In [ ]:
# Convert the NumPy arrays back to PyTorch tensors (and move to GPU if available)
train_images = torch.tensor(X_train_numpy, dtype=torch.float32).unsqueeze(1).to(device)
test_images = torch.tensor(X_test_numpy, dtype=torch.float32).unsqueeze(1).to(device)
train_labels = torch.tensor(y_train_numpy, dtype=torch.float32).to(device)
test_labels = torch.tensor(y_test_numpy, dtype=torch.float32).to(device)

In [ ]:
train_images.shape

torch.Size([27800, 1, 64, 64])

In [ ]:
# Create DataLoader for training and testing datasets
train_dataset = TensorDataset(train_images, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
test_dataset = TensorDataset(test_images, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Load a pre-trained ResNet model
resnet50 = models.resnet50(pretrained=True)

# Change the input layer to accept single-channel (grayscale) images
resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=2, stride=1, padding=1, bias=False)

# Freeze all layers except the final classification layer
for param in resnet50.parameters():
    param.requires_grad = False
num_features = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_features, 2)  # 2 output classes for binary classification

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.fc.parameters(), lr=0.001)

resnet50.to(device)

In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs=inputs.to(device)
        labels = labels.type(torch.LongTensor).to(device)
        labels.unsqueeze(1);
        labels.to(device)

        optimizer.zero_grad()

        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


KeyboardInterrupt: ignored

In [ ]:
resnet50.eval()  # Set the model to evaluation mode

# Initialize variables to store predictions and ground truth labels
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        labels.unsqueeze(1)
        outputs = resnet50(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the class with the highest probability
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7446043165467626


# RESNET 101

In [ ]:
# Convert the NumPy arrays back to PyTorch tensors (and move to GPU if available)
train_images = torch.tensor(X_train_numpy, dtype=torch.float32).unsqueeze(1).to(device)
test_images = torch.tensor(X_test_numpy, dtype=torch.float32).unsqueeze(1).to(device)
train_labels = torch.tensor(y_train_numpy, dtype=torch.float32).to(device)
test_labels = torch.tensor(y_test_numpy, dtype=torch.float32).to(device)

In [ ]:
train_images.shape

torch.Size([27800, 1, 64, 64])

In [ ]:
# Create DataLoader for training and testing datasets
train_dataset = TensorDataset(train_images, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
test_dataset = TensorDataset(test_images, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Load a pre-trained ResNet model
model = models.resnet101(pretrained=True)

# Change the input layer to accept single-channel (grayscale) images
model.conv1 = nn.Conv2d(1, 64, kernel_size=2, stride=1, padding=1, bias=False)

# Freeze all layers except the final classification layer
for param in model.parameters():
    param.requires_grad = False
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 output classes for binary classification

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 192MB/s]


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

model.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs=inputs.to(device)
        labels = labels.type(torch.LongTensor).to(device)
        labels.unsqueeze(1);
        labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/10, Loss: 0.46861652753844224
Epoch 2/10, Loss: 0.44498347243244823
Epoch 3/10, Loss: 0.4319429338081248
Epoch 4/10, Loss: 0.4309192004220258
Epoch 5/10, Loss: 0.41084599537802785
Epoch 6/10, Loss: 0.41106968881483047
Epoch 7/10, Loss: 0.4067705860929467
Epoch 8/10, Loss: 0.4054908898880896
Epoch 9/10, Loss: 0.3954639312381437
Epoch 10/10, Loss: 0.39955357847986067


In [ ]:
model.eval()  # Set the model to evaluation mode

# Initialize variables to store predictions and ground truth labels
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        labels.unsqueeze(1)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the class with the highest probability
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8241726618705036


# RESNET 152

In [ ]:
# Convert the NumPy arrays back to PyTorch tensors (and move to GPU if available)
train_images = torch.tensor(X_train_numpy, dtype=torch.float32).unsqueeze(1).to(device)
test_images = torch.tensor(X_test_numpy, dtype=torch.float32).unsqueeze(1).to(device)
train_labels = torch.tensor(y_train_numpy, dtype=torch.float32).to(device)
test_labels = torch.tensor(y_test_numpy, dtype=torch.float32).to(device)

In [ ]:
train_images.shape

torch.Size([27800, 1, 64, 64])

In [ ]:
# Create DataLoader for training and testing datasets
train_dataset = TensorDataset(train_images, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
test_dataset = TensorDataset(test_images, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Load a pre-trained ResNet model
model = models.resnet152(pretrained=True)

# Change the input layer to accept single-channel (grayscale) images
model.conv1 = nn.Conv2d(1, 64, kernel_size=2, stride=1, padding=1, bias=False)

# Freeze all layers except the final classification layer
for param in model.parameters():
    param.requires_grad = False
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 output classes for binary classification

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

model.to(device)

In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs=inputs.to(device)
        labels = labels.type(torch.LongTensor).to(device)
        labels.unsqueeze(1);
        labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/10, Loss: 0.4770640081997159
Epoch 2/10, Loss: 0.44823785853262466
Epoch 3/10, Loss: 0.4370513489540619
Epoch 4/10, Loss: 0.4249473897991027
Epoch 5/10, Loss: 0.42786318499281983
Epoch 6/10, Loss: 0.4150416051456651
Epoch 7/10, Loss: 0.41385869680120424
Epoch 8/10, Loss: 0.40410730713427545
Epoch 9/10, Loss: 0.40794318753227127
Epoch 10/10, Loss: 0.40204832681440783


In [ ]:
model.eval()  # Set the model to evaluation mode

# Initialize variables to store predictions and ground truth labels
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        labels.unsqueeze(1)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the class with the highest probability
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8287769784172662
